In [1]:
# ## 1) Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# ## 2) Install / update libraries (Sentence‑Transformers)
!pip install -q -U "transformers>=4.51.0" "sentence-transformers>=2.7.0" accelerate bitsandbytes tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# ## 3) Load relation list
import os, math, numpy as np
from tqdm import tqdm

REL_PATH = "/content/drive/MyDrive/HotpotQA_snapshot/unique_relations.txt"
assert os.path.exists(REL_PATH), REL_PATH

with open(REL_PATH, 'r', encoding='utf-8') as f:
    relations_raw = [line.rstrip('\n') for line in f if line.strip()]

N = len(relations_raw)
print(f"Relations loaded: {N:,}")


Relations loaded: 139,253


In [4]:
# ## 4) Prepare texts for embedding
relations_for_embed = [r.replace('_', ' ') for r in relations_raw]
assert len(relations_for_embed) == N

In [5]:
# ## 5) Load Qwen3‑Embedding‑8B 
from sentence_transformers import SentenceTransformer

MODEL_ID  = "Qwen/Qwen3-Embedding-8B"

print("Loading SentenceTransformer model …")
model = SentenceTransformer(
    MODEL_ID,
    model_kwargs={"device_map": "auto", "torch_dtype": "auto"},
    tokenizer_kwargs={"padding_side": "left"},
)
EMBED_DIM  = model.get_sentence_embedding_dimension()   # باید 4096 باشد
BATCH_SIZE = 8
model.eval()

print("Model embed dim:", EMBED_DIM)


Loading SentenceTransformer model …


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

Model embed dim: 4096


In [6]:
# ## 6) Allocate float16 memmap (≈ ۱٫۱ GB)
EMB_PATH = "/content/drive/MyDrive/HotpotQA_snapshot/unique_relations_emb_4096.fp16.memmap"
emb_mem  = np.memmap(EMB_PATH, mode="w+", dtype="float16", shape=(N, EMBED_DIM))
print(f"Memmap ↔ {EMB_PATH} | shape {(N, EMBED_DIM)}")


Memmap ↔ /content/drive/MyDrive/HotpotQA_snapshot/unique_relations_emb_4096.fp16.memmap | shape (139253, 4096)


In [7]:
# ## 7) Batch‑encode & store embeddings
import torch

for start in tqdm(range(0, N, BATCH_SIZE), total=math.ceil(N / BATCH_SIZE), desc="Embedding"):
    batch_texts = relations_for_embed[start : start + BATCH_SIZE]

    emb = model.encode(
        batch_texts,
        batch_size      = BATCH_SIZE,
        convert_to_numpy= True,    
        normalize_embeddings = True,
        show_progress_bar   = False,
    ).astype("float16")

    emb_mem[start : start + len(batch_texts)] = emb

    if (start // BATCH_SIZE) % 50 == 0:
        emb_mem.flush()
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

emb_mem.flush()
del emb_mem
print("✔️  All relation embeddings stored.")


Embedding: 100%|██████████| 17407/17407 [25:06<00:00, 11.55it/s]

✔️  All relation embeddings stored.


In [8]:
# ## 8) Save relation order for index mapping
ORDER_PATH = "/content/drive/MyDrive/HotpotQA_snapshot/unique_relations_ordered.txt"
with open(ORDER_PATH, 'w', encoding='utf-8') as f:
    f.writelines(rel + '\n' for rel in relations_raw)

print("Order saved →", ORDER_PATH)


Order saved → /content/drive/MyDrive/HotpotQA_snapshot/unique_relations_ordered.txt


In [9]:
# ## 9) Sanity‑check a random row
emb_check = np.memmap(EMB_PATH, mode='r', dtype='float16', shape=(N, EMBED_DIM))

idx = 1234            # یا هر ایندکس دلخواه
print("Relation:", relations_raw[idx])
print("For‑embed:", relations_for_embed[idx])
print("Sample dims:", emb_check[idx, :5])
print("Norm ≈", np.linalg.norm(emb_check[idx]))


Relation: acquired_on
For‑embed: acquired on
Sample dims: [ 0.02087   0.0221   -0.0058   -0.03052   0.012695]
Norm ≈ 0.999
